Potential field approach:
suppose that the robot is moving from a starting point to a goal point, using Energy concept, we can say that 
the robot will moves to a lower energy configuration. the aim is to minimize the energy by following the negative gradient of the potential energy function: 
$$
\nabla U(q) = \left[ \frac{\partial U}{\partial q_1}(q),\ \dots,\ \frac{\partial U}{\partial q_m}(q) \right]
$$
$$
U(q) = U_{\text{att}}(q) + U_{\text{rep}}(q)
$$
where: 

$U_{\text{att}}$ stands for attractive potential which encourage moving to the goal.

$U_{\text{rep}}$ stands for repulsive potential which encourage avoiding obstacles.

First let's calculate $U_{\text{att}}$ :

to achieve a smooth movement where the robot gradually slows down near the obstacles and move with constant speed otherwise, we can consider to Potential functions:
$$
U_{\text{quadratic}} = \frac{1}{2}*k*norm(q-q_g)^2
$$

$$
U_{\text{Conical}} = k*norm(q-q_g)
$$

$$
U_{\text{att}} = U_{\text{Conical}} + U_{\text{quadratic}}
$$

$$
F_{\text{att}} = - \nabla U_{\text{att}}
$$

now let's calculate repulsive potential field: 
$$
U_{\text{rep}} = 
\begin{cases} 
\frac{1}{2} K_{\text{rep}} \left(\frac{1}{d} - \frac{1}{d_0}\right)^2 & \text{if } d \leq d_0 \\
0 & \text{otherwise}
\end{cases}
$$
$$
F_{\text{rep}} = - \nabla U_{\text{rep}}
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import plotly.graph_objects as go
import roboticstoolbox as rtb
import spatialmath.base as sb
import pybullet as p
import pybullet_data
import time
import numpy as np
from pybullet_planning import plan_joint_motion, enable_gravity, connect
from pybullet_planning import set_joint_positions
import time
from pybullet_planning import (
    get_sample_fn, get_extend_fn, get_collision_fn, get_distance_fn
)


In [ ]:
# define the robot using robotics toolbox to get inverse kinemaics
robot = rtb.models.DH.Panda()


## Defining 3D potential field

In [ ]:
# Attractive field
def U_att(X, Y, Z, x_goal, y_goal, z_goal, D_goal, K1, K2):
    U = np.zeros_like(X)
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            for k in range(X.shape[2]):
                d = np.linalg.norm([X[i,j,k] - x_goal, Y[i,j,k] - y_goal, Z[i,j,k] - z_goal])
                if d <= D_goal:
                    U[i,j,k] = K1 * d
                else:
                    U[i,j,k] = 0.5 * K2 * d**2
    return np.clip(U, -10000, 10000)

In [ ]:
# Repulsive field
def U_rep(X, Y, Z, obstacles, K_rep, d0):
    U = np.zeros_like(X)
    for (obs_x, obs_y, obs_z) in obstacles:
        d = np.sqrt((X - obs_x)**2 + (Y - obs_y)**2 + (Z - obs_z)**2)
        with np.errstate(divide='ignore'):
            U += np.where(d <= d0, 0.5 * K_rep * (1/d - 1/d0)**2, 0)
    return np.clip(U, -1000, 1000)

In [ ]:
# Gradient
def numerical_gradient(U, h=0.5):
    grad_x = np.zeros_like(U)
    grad_y = np.zeros_like(U)
    grad_z = np.zeros_like(U)

    grad_x[1:-1,:,:] = np.clip((U[2:,:,:] - U[:-2,:,:]) / (2*h) , -1000, 1000)
    grad_y[:,1:-1,:] = np.clip((U[:,2:,:] - U[:,:-2,:]) / (2*h), -1000, 1000)
    grad_z[:,:,1:-1] = np.clip((U[:,:,2:] - U[:,:,:-2]) / (2*h), -1000, 1000)
    grad_x[0,:,:] = np.clip((U[1,:,:] - U[0,:,:]) / h, -1000, 1000)
    grad_x[-1,:,:] = np.clip((U[-1,:,:] - U[-2,:,:]) / h, -1000, 1000)
    grad_y[:,0,:] = np.clip((U[:,1,:] - U[:,0,:]) / h, -1000, 1000)
    grad_y[:,-1,:] = np.clip((U[:,-1,:] - U[:,-2,:]) / h, -1000, 1000)
    grad_z[:,:,0] = np.clip((U[:,:,1] - U[:,:,0]) / h, -1000, 1000)
    grad_z[:,:,-1] = np.clip((U[:,:,-1] - U[:,:,-2]) / h, -1000, 1000)
    return grad_x, grad_y, grad_z


In [ ]:
def select_next_move_by_force_3d(pmap, ix, iy, iz, motion):
    max_force = -float("inf")
    best_ix, best_iy, best_iz = ix, iy, iz
    current_potential = pmap[ix, iy, iz]

    for dx, dy, dz in motion:
        nx, ny, nz = ix + dx, iy + dy, iz + dz
        if (0 <= nx < pmap.shape[0] and
            0 <= ny < pmap.shape[1] and
            0 <= nz < pmap.shape[2]):
            neighbor_potential = pmap[nx, ny, nz]
            delta_p = current_potential - neighbor_potential
            step_length = np.sqrt(dx*dx + dy*dy + dz*dz)
            force = delta_p / step_length if step_length != 0 else 0
            if force > max_force:
                max_force = force
                best_ix, best_iy, best_iz = nx, ny, nz

    return best_ix, best_iy, best_iz

In [ ]:
def plan_path_3d(start, goal, U_total, X, Y, Z, step_size=0.05, max_steps=10000):
    path = [start.copy()]
    current_pos = start.copy()

    def pos_to_index(pos):
        xi = np.argmin(np.abs(X[:,0,0] - pos[0]))
        yi = np.argmin(np.abs(Y[0,:,0] - pos[1]))
        zi = np.argmin(np.abs(Z[0,0,:] - pos[2]))
        return xi, yi, zi

    motion = [
        (1,0,0), (-1,0,0), (0,1,0), (0,-1,0), (0,0,1), (0,0,-1),
        (1,1,0), (1,-1,0), (-1,1,0), (-1,-1,0),
        (1,0,1), (1,0,-1), (-1,0,1), (-1,0,-1),
        (0,1,1), (0,1,-1), (0,-1,1), (0,-1,-1),
        (1,1,1), (1,1,-1), (1,-1,1), (1,-1,-1),
        (-1,1,1), (-1,1,-1), (-1,-1,1), (-1,-1,-1)
    ]

    for _ in range(max_steps):
        ix, iy, iz = pos_to_index(current_pos)
        next_ix, next_iy, next_iz = select_next_move_by_force_3d(U_total, ix, iy, iz, motion)

        if (next_ix, next_iy, next_iz) == (ix, iy, iz):
            print("Trapped in local minimum.")
            break

        next_pos = np.array([X[next_ix, 0, 0], Y[0, next_iy, 0], Z[0, 0, next_iz]])
        current_pos = next_pos
        path.append(current_pos.copy())

        if np.linalg.norm(current_pos - goal) < step_size/10:
            print("Reached the goal!")
            break
    return np.array(path)

In [ ]:
# Setting the environment 
x = np.arange(-1, 1, 0.05)
y = np.arange(-1, 1, 0.05)
z = np.arange(-1, 1, 0.05)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

start_pos = np.array([-0.5, -0.5, 0.5])
goal_pos = np.array([0.7, 0.7, 0.7])

T_start = sb.transl(start_pos)
T_goal = sb.transl(goal_pos)
q_LM_start = robot.ikine_LM(T_start).q  # Levenberg-Marquardt
q_LM_goal = robot.ikine_LM(T_goal).q
# Connect to GUI
client = connect(use_gui=True)
p.setAdditionalSearchPath(pybullet_data.getDataPath())

# Load plane and robot
plane = p.loadURDF("plane.urdf")
# robot_id = p.loadURDF("kuka_iiwa/model.urdf", useFixedBase=True)
robot_id = p.loadURDF("franka_panda/panda.urdf", useFixedBase=True)

# Enable gravity
enable_gravity()

# Create a box obstacle in front of the robot
import random

# Parameters
radius = 0.05  # Small radius
obstacles = []
positions = []
while len(obstacles) == 0:
    # Generate 20 random positions within a defined workspace
    for _ in range(50):
        x = random.uniform(-0.7, 0.7)
        y = random.uniform(-0.6, 0.6)
        z = random.uniform(0.2, 0.8)
        
        positions.append([x, y, z])

    # Create visual and collision shapes once (for efficiency)
    obstacle_visual = p.createVisualShape(
        shapeType=p.GEOM_SPHERE,
        radius=radius,
        rgbaColor=[1, 0, 0, 0.8]  # Red spheres
    )

    obstacle_collision = p.createCollisionShape(
        shapeType=p.GEOM_SPHERE,
        radius=radius
    )

    # Create each obstacle at a different position
    for pos in positions:
        obstacle_id = p.createMultiBody(
            baseMass=0,
            baseCollisionShapeIndex=obstacle_collision,
            baseVisualShapeIndex=obstacle_visual,
            basePosition=pos
        )

        obstacles.append(obstacle_id)

q_start = q_LM_start
q_goal = q_LM_goal
joint_indices= [0,1,2,3,4,5,6]
set_joint_positions(robot_id, joint_indices, q_start) 


In [ ]:
D_goal = 0.05
K1, K2 = 0.5, 0.2
K_rep = 2
d0 = 0.05

# Compute potentials
U_att_total = U_att(X, Y, Z, goal_pos[0], goal_pos[1], goal_pos[2], D_goal, K1, K2)
U_rep_total = U_rep(X, Y, Z, positions, K_rep, d0)
U_total = U_att_total + U_rep_total

# Compute gradient and force magnitude
F_x, F_y, F_z = numerical_gradient(U_total, h=0.05)
F_x *= -1
F_y *= -1
F_z *= -1
F_total = np.sqrt(F_x**2 + F_y**2 + F_z**2)

# Plan the path
path = plan_path_3d(start_pos, goal_pos, U_total, X, Y, Z)


In [ ]:
fig = go.Figure()
levels = [0.2, 0.4, 0.6, 0.8]
max_val = np.max(F_total)
for level in levels:
    fig.add_trace(go.Isosurface(
        x=X.flatten(),
        y=Y.flatten(),
        z=Z.flatten(),
        value=F_total.flatten(),
        isomin=level * max_val,
        isomax=max_val,
        surface_count=1,
        colorscale='Blues',
        opacity=0.3,
        caps=dict(x_show=False, y_show=False, z_show=False),
        showscale=False,
        name=f'Iso {level*100:.0f}%'
    ))

# Add start and goal points
fig.add_trace(go.Scatter3d(
    x=[start_pos[0]],
    y=[start_pos[1]],
    z=[start_pos[2]],
    mode='markers',
    marker=dict(color='blue', size=8),
    name='Start'
))

fig.add_trace(go.Scatter3d(
    x=[goal_pos[0]],
    y=[goal_pos[1]],
    z=[goal_pos[2]],
    mode='markers',
    marker=dict(color='green', size=8, symbol='circle'),
    name='Goal'
))

# Add obstacles
obs_x, obs_y, obs_z = zip(*positions)
fig.add_trace(go.Scatter3d(
    x=obs_x,
    y=obs_y,
    z=obs_z,
    mode='markers',
    marker=dict(color='red', size=0.6, symbol='x'),
    name='Obstacles'
))

# Add planned path
fig.add_trace(go.Scatter3d(
    x=path[:,0],
    y=path[:,1],
    z=path[:,2],
    mode='lines+markers',
    marker=dict(size=3, color='yellow'),
    line=dict(color='red', width=4),
    name='Path'
))

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ),
    title='3D Potential Field Path Planning',
    width=900, height=800
)
fig.write_html("path_planning_3d.html")
fig.show()


## converting the path to configuration space and run the simulation in Pybullet

In [ ]:
q_path = []
q_path.append(q_LM_start)
for i in range(1,len(path)):
    T_end = sb.transl(path[i])
    q_LM = robot.ikine_LM(T_end).q 
    q_path.append(q_LM)

q_path.append(q_LM_goal)


In [ ]:
set_joint_positions(robot_id, joint_indices, q_start) 
for i in range(len(q_path)):
    qq= np.asarray(q_path)[i]
    set_joint_positions(robot_id, joint_indices, qq)
    time.sleep(0.05)